# 論文 10：用於圖像識別的深度殘差學習
## Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun (2015)

### ResNet：跳躍連接使非常深的網路成為可能

ResNet 引入了殘差連接，使得訓練超過 100 層的網路成為可能。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## 問題：深度網路的退化

在 ResNet 之前，添加更多層實際上會使網路變差（不是因為過擬合，而是優化困難）。

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

class PlainLayer:
    """標準神經網路層"""
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(output_size, input_size) * np.sqrt(2.0 / input_size)
        self.b = np.zeros((output_size, 1))
    
    def forward(self, x):
        self.x = x
        self.z = np.dot(self.W, x) + self.b
        self.a = relu(self.z)
        return self.a
    
    def backward(self, dout):
        da = dout * relu_derivative(self.z)
        self.dW = np.dot(da, self.x.T)
        self.db = np.sum(da, axis=1, keepdims=True)
        dx = np.dot(self.W.T, da)
        return dx

class ResidualBlock:
    """帶跳躍連接的殘差區塊：y = F(x) + x"""
    def __init__(self, size):
        self.layer1 = PlainLayer(size, size)
        self.layer2 = PlainLayer(size, size)
    
    def forward(self, x):
        self.x = x
        
        # 殘差路徑 F(x)
        out = self.layer1.forward(x)
        out = self.layer2.forward(out)
        
        # 跳躍連接：F(x) + x
        self.out = out + x
        return self.out
    
    def backward(self, dout):
        # 梯度通過兩條路徑流動
        # 跳躍連接提供直接路徑
        dx_residual = self.layer2.backward(dout)
        dx_residual = self.layer1.backward(dx_residual)
        
        # 總梯度：殘差路徑 + 跳躍連接
        dx = dx_residual + dout  # 這是關鍵！
        return dx

print("ResNet 元件已初始化")

## 建構普通網路 vs ResNet

In [ ]:
class PlainNetwork:
    """沒有跳躍連接的普通深度網路"""
    def __init__(self, input_size, hidden_size, num_layers):
        self.layers = []
        
        # 第一層
        self.layers.append(PlainLayer(input_size, hidden_size))
        
        # 隱藏層
        for _ in range(num_layers - 2):
            self.layers.append(PlainLayer(hidden_size, hidden_size))
        
        # 輸出層
        self.layers.append(PlainLayer(hidden_size, input_size))
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, dout):
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

class ResidualNetwork:
    """帶殘差連接的深度網路"""
    def __init__(self, input_size, hidden_size, num_blocks):
        # 投影到隱藏大小
        self.input_proj = PlainLayer(input_size, hidden_size)
        
        # 殘差區塊
        self.blocks = [ResidualBlock(hidden_size) for _ in range(num_blocks)]
        
        # 投影回輸出
        self.output_proj = PlainLayer(hidden_size, input_size)
    
    def forward(self, x):
        x = self.input_proj.forward(x)
        for block in self.blocks:
            x = block.forward(x)
        x = self.output_proj.forward(x)
        return x
    
    def backward(self, dout):
        dout = self.output_proj.backward(dout)
        for block in reversed(self.blocks):
            dout = block.backward(dout)
        dout = self.input_proj.backward(dout)
        return dout

# 建立網路
input_size = 16
hidden_size = 16
depth = 10

plain_net = PlainNetwork(input_size, hidden_size, depth)
resnet = ResidualNetwork(input_size, hidden_size, depth)

print(f"建立了 {depth} 層的普通網路")
print(f"建立了 {depth} 個殘差區塊的 ResNet")

## 展示梯度流動

關鍵優勢：梯度通過跳躍連接更容易流動

In [ ]:
def measure_gradient_flow(network, name):
    """測量不同深度的梯度大小"""
    # 隨機輸入
    x = np.random.randn(input_size, 1)
    
    # 前向傳遞
    output = network.forward(x)
    
    # 建立梯度信號
    dout = np.ones_like(output)
    
    # 反向傳遞
    network.backward(dout)
    
    # 收集梯度大小
    grad_norms = []
    
    if isinstance(network, PlainNetwork):
        for layer in network.layers:
            grad_norm = np.linalg.norm(layer.dW)
            grad_norms.append(grad_norm)
    else:  # ResNet
        grad_norms.append(np.linalg.norm(network.input_proj.dW))
        for block in network.blocks:
            grad_norm1 = np.linalg.norm(block.layer1.dW)
            grad_norm2 = np.linalg.norm(block.layer2.dW)
            grad_norms.append(np.mean([grad_norm1, grad_norm2]))
        grad_norms.append(np.linalg.norm(network.output_proj.dW))
    
    return grad_norms

# 測量兩個網路的梯度流動
plain_grads = measure_gradient_flow(plain_net, "普通網路")
resnet_grads = measure_gradient_flow(resnet, "ResNet")

# 繪製比較
plt.figure(figsize=(12, 5))
plt.plot(range(len(plain_grads)), plain_grads, 'o-', label='普通網路', linewidth=2)
plt.plot(range(len(resnet_grads)), resnet_grads, 's-', label='ResNet', linewidth=2)
plt.xlabel('層深度（越深 →）')
plt.ylabel('梯度大小')
plt.title('梯度流動：ResNet vs 普通網路')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\n普通網路 - 第一層梯度：{plain_grads[0]:.6f}")
print(f"普通網路 - 最後層梯度：{plain_grads[-1]:.6f}")
print(f"梯度比（第一/最後）：{plain_grads[0]/plain_grads[-1]:.2f}x\n")

print(f"ResNet - 第一層梯度：{resnet_grads[0]:.6f}")
print(f"ResNet - 最後層梯度：{resnet_grads[-1]:.6f}")
print(f"梯度比（第一/最後）：{resnet_grads[0]/resnet_grads[-1]:.2f}x")

print(f"\nResNet 維持梯度流動好 {(plain_grads[0]/plain_grads[-1]) / (resnet_grads[0]/resnet_grads[-1]):.1f} 倍！")

## 視覺化學習到的表示

In [ ]:
# 生成合成的類圖像資料
def generate_patterns(num_samples=100, size=8):
    """生成簡單的 2D 模式"""
    X = []
    y = []
    
    for i in range(num_samples):
        pattern = np.zeros((size, size))
        
        if i % 3 == 0:
            # 水平線
            pattern[2:3, :] = 1
            label = 0
        elif i % 3 == 1:
            # 垂直線
            pattern[:, 3:4] = 1
            label = 1
        else:
            # 對角線
            np.fill_diagonal(pattern, 1)
            label = 2
        
        # 添加雜訊
        pattern += np.random.randn(size, size) * 0.1
        
        X.append(pattern.flatten())
        y.append(label)
    
    return np.array(X), np.array(y)

X, y = generate_patterns(num_samples=30, size=4)

# 視覺化範例模式
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for i, ax in enumerate(axes):
    sample = X[i].reshape(4, 4)
    ax.imshow(sample, cmap='gray')
    ax.set_title(f'模式類型 {y[i]}')
    ax.axis('off')
plt.show()

print(f"生成了 {len(X)} 個模式樣本")

## 恆等映射：核心洞見

**關鍵洞見**：如果恆等映射是最優的，殘差應該學習 F(x) = 0，這比學習 H(x) = x 更容易

In [ ]:
# 展示恆等映射
x = np.random.randn(hidden_size, 1)

# 初始化殘差區塊
block = ResidualBlock(hidden_size)

# 如果權重接近零，F(x) ≈ 0
block.layer1.W *= 0.001
block.layer2.W *= 0.001

# 前向傳遞
output = block.forward(x)

# 檢查輸出是否約等於輸入（恆等）
identity_error = np.linalg.norm(output - x)

print("恆等映射展示：")
print(f"輸入範數：{np.linalg.norm(x):.4f}")
print(f"輸出範數：{np.linalg.norm(output):.4f}")
print(f"恆等誤差 ||F(x) + x - x||：{identity_error:.6f}")
print(f"\n當權重接近零時，殘差區塊 ≈ 恆等函數！")

# 視覺化
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(x.flatten(), 'o-', label='輸入 x', alpha=0.7)
plt.plot(output.flatten(), 's-', label='輸出 (x + F(x))', alpha=0.7)
plt.xlabel('維度')
plt.ylabel('值')
plt.title('恆等映射：輸出 ≈ 輸入')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
residual = output - x
plt.bar(range(len(residual)), residual.flatten())
plt.xlabel('維度')
plt.ylabel('殘差 F(x)')
plt.title('學習到的殘差 ≈ 0')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 比較不同網路深度

In [ ]:
def test_depth_scaling():
    """測試梯度流動如何隨深度變化"""
    depths = [5, 10, 20, 30, 40]
    plain_ratios = []
    resnet_ratios = []
    
    for depth in depths:
        # 建立網路
        plain = PlainNetwork(input_size, hidden_size, depth)
        res = ResidualNetwork(input_size, hidden_size, depth)
        
        # 測量梯度
        plain_grads = measure_gradient_flow(plain, "Plain")
        res_grads = measure_gradient_flow(res, "ResNet")
        
        # 計算比率（第一層/最後層梯度）
        plain_ratio = plain_grads[0] / (plain_grads[-1] + 1e-10)
        res_ratio = res_grads[0] / (res_grads[-1] + 1e-10)
        
        plain_ratios.append(plain_ratio)
        resnet_ratios.append(res_ratio)
    
    # 繪製
    plt.figure(figsize=(10, 6))
    plt.plot(depths, plain_ratios, 'o-', label='普通網路', linewidth=2, markersize=8)
    plt.plot(depths, resnet_ratios, 's-', label='ResNet', linewidth=2, markersize=8)
    plt.xlabel('網路深度')
    plt.ylabel('梯度比（第一/最後層）')
    plt.title('梯度流動隨深度的退化')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.show()
    
    print("\n梯度比（第一/最後）- 越高 = 梯度流動越差：")
    for i, d in enumerate(depths):
        print(f"深度 {d:2d}：普通={plain_ratios[i]:8.2f}，ResNet={resnet_ratios[i]:6.2f} "
              f"（ResNet 好 {plain_ratios[i]/resnet_ratios[i]:.1f} 倍）")

test_depth_scaling()

## 關鍵要點

### 退化問題：
- 給普通網路添加更多層會損害效能
- **不是**因為過擬合（訓練誤差也增加）
- 是因為優化困難：梯度消失/爆炸

### ResNet 解決方案：跳躍連接
```
y = F(x, {Wi}) + x
```

**不再學習**：H(x) = 期望映射  
**而是學習殘差**：F(x) = H(x) - x  
**然後**：H(x) = F(x) + x

### 為什麼有效：
1. **恆等映射更容易**：如果最優映射是恆等，學習 F(x) = 0 比學習 H(x) = x 更容易
2. **梯度高速公路**：跳躍連接提供直接的梯度路徑
3. **加性梯度流動**：梯度同時通過殘差和跳躍路徑
4. **無額外參數**：跳躍連接無需參數

### 影響：
- 使 152 層網路成為可能（之前限制為約 20 層）
- 贏得 ImageNet 2015（3.57% top-5 錯誤率）
- 成為標準架構模式
- 啟發了變體：DenseNet、ResNeXt 等

### 數學洞見：
損失 L 對前面層的梯度：
```
∂L/∂x = ∂L/∂y * (∂F/∂x + ∂x/∂x) = ∂L/∂y * (∂F/∂x + I)
```
`+ I` 項確保梯度始終能流動！